Connect to TigerGraph and TigerGraph GraphRAG

In [1]:
from pyTigerGraph import TigerGraphConnection

conn = TigerGraphConnection(
    host="https://YOUR_DB_ADDRESS", 
    graphname="Transaction_Fraud", 
    username="YOUR_DB_USERNAME", 
    password="YOUR_DB_PASSWORD"
)

conn.getToken()

conn.ai.configureGraphRAGHost(hostname="http://GRAPHRAG_ADDRESS")

List of questions:
1. Find all transactions, from 2/1/21 to 5/1/21, above average amount for that card. Sort the results.

2. Provide more details about the transaction 871054.

3. What is the average transaction amount for the card 4039101933538921 used in this transaction over the past 6 months?

4. What do we know about the merchant fraud_Kuhn LLC involved in the transaction 871054?

5. Are there any activities of the user associated with the transaction 871054 that might be interesting to look into for further investigation? 

In [2]:
import json
import time
import requests
from requests.auth import HTTPBasicAuth

num_runs = 5

questions = [
    "Find all transactions, from 2/1/21 to 5/1/21 above average amount for that card. Sort the results.",
    "Provide more details about the transaction 871054.",
    "What is the average transaction amount for the card used 4039101933538921 in this transaction over the past 6 months?",
    "Provide more details about the merchant fraud_Kuhn LLC involved in the transaction 871054",
    "Are there any activities of the user associated with the transaction 871054 that might be interesting to look into for further investigation?"
]

# Dictionary to store execution times for each question
question_execution_times = {question: [] for question in questions}

for run_number in range(1, num_runs + 1):
    print(f"Run {run_number}:")
    for question in questions:
        start_time = time.time()
        payload = {
            "query": question
        }

        res = requests.post("http://GRAPHRAG_ADDRESS/Transaction_Fraud/query_with_history", 
                            json=payload, 
                            auth=HTTPBasicAuth(conn.username, conn.password)
        )
        print (json.loads(res.text).get("natural_language_response", "no answer"))
        end_time = time.time()

        execution_time = end_time - start_time
        question_execution_times[question].append(execution_time)

    print("")

for question, times in question_execution_times.items():
    average_time = sum(times) / num_runs
    print(f"Average execution time for question '{question}' over {num_runs} runs: {average_time:.4f} seconds")

Run 1:
The transactions from 2/1/21 to 5/1/21 above average amount for that card are as follows: Transaction with id 39843 on 2021-04-17 17:36:29 with amount 7855.36, Transaction with id 871054 on 2021-02-02 00:00:00 with amount 5321.52, Transaction with id 40831 on 2021-04-24 23:45:10 with amount 2934.13, Transaction with id 34336 on 2021-04-24 12:24:53 with amount 4935.42, Transaction with id 473824 on 2021-04-18 14:29:51 with amount 3713.09, Transaction with id 7815 on 2021-04-17 16:05:56 with amount 6044.24, Transaction with id 769 on 2021-04-17 23:31:10 with amount 2802.55.
Transaction 871054 is a payment transaction that occurred at 2021-02-02 00:00:00. The amount of the transaction was 5321.52. It is marked as a fraudulent transaction. There are no repeated cards, merchant transactions, or common code transactions associated with this transaction. The transaction does not have any merchant category, occupation, or gender associated with it. The age and city population related to